In [1]:
import sys
from pathlib import Path
parent_dir = Path.cwd().parent
sys.path.append(str(parent_dir))

# Digital Signature, Transaction

In [ ]:
from src.digital_signature.ecc import secp256k1 # NOTE: There are automated tests for this module
from src.blockchain.block import Transaction
import random

In [3]:
private_key  = random.randint(1, secp256k1.n-1)
public_key = secp256k1.scalar_multiply(private_key, secp256k1.G)
private_key, public_key  

(81887582542687856176065109036894280514370813961086777460981828027979094873831,
 (82656216790249264661209255248079511074180901952758679129451597651865710098962,
  334414800223850064553906847647671735838214391190049804644380619439251544653))

In [4]:
tx = Transaction(sender="Miner Micky", receiver="Miner Moe", amount=100)

In [5]:
tx.hash_transaction()

91469887455764377586877807764597904145762171308828855680586952901254648126450

In [6]:
tx.sign(private_key)
tx.signature

(40399957570093825966803356468860600440685294084868110689532086954465302911264,
 97664644634066426596605947201771500567158417885312160533335701209264856945983)

In [7]:
is_valid = tx.verify(public_key)
is_valid

True

# Block

In [8]:
from src.blockchain.block import Block

In [9]:
tx2 = Transaction(sender="Miner Mike", receiver="Miner Micky", amount=50)
tx3 = Transaction(sender="Miner Mikey", receiver="Miner Micky", amount=60)

our_block = Block(index=0, previous_hash="0", data=[tx, tx2, tx3])

In [10]:
our_block.hash

'b83fa86a7ba0877fa51f44e2780ab3400bf006ecc3348e0f09c37cf9edcbbc39'

In [11]:
our_block.data

In [12]:
def hash_criterion(hash: str):
    return hash.startswith("00")

In [13]:
# PoW
our_block.mine(hash_criterion)

'0047692198739e72807048cccede01262c722f2ab18b3cb01d089385e9f30e0f'

# Blockchain

In [14]:
from src.blockchain.blockchain import Blockchain

In [15]:
our_blockchain = Blockchain(hash_criterion)

In [16]:
# Genesis block:
our_blockchain.chain[0].data

[]

# Network

In [17]:
from src.blockchain.network import Network

In [18]:
our_network = Network(blockchain=our_blockchain)

In [19]:
our_network.add_node("Miner Micky")
our_network.add_node("Miner Moe")
our_network.add_node("Miner Jack")

In [20]:
our_network.nodes

{'Miner Micky': <src.blockchain.node.Node at 0x18c06b972c0>,
 'Miner Moe': <src.blockchain.node.Node at 0x18c06bcad80>,
 'Miner Jack': <src.blockchain.node.Node at 0x18c06bea420>}

In [21]:
our_network.public_keys

{'Miner Micky': (41622633588637395064837534632899103177563255021810681339942519219329431454502,
  11339119106049453601251128794006752414275810446194026569494748432985708591532),
 'Miner Moe': (39404460190203425395014273408697332962575588123573101922508858119537741014037,
  66668214048429706225999017230721584943820025352067835298249834981604891742727),
 'Miner Jack': (104063203318913841572802214968800797595460737852747763654634590208507961887739,
  74392400257205263102671162614944821931143060614409424705746814185051811570221)}

In [22]:
our_network.mine_block_turn_based(block=our_block, max_attempts_per_node=100)

Starting turn-based mining...
Miner Micky is attempting to mine...
Miner Moe is attempting to mine...
Miner Jack is attempting to mine...
No node found a valid block. Restarting round...
Miner Micky is attempting to mine...
Miner Moe is attempting to mine...
Miner Jack is attempting to mine...
No node found a valid block. Restarting round...
Miner Micky is attempting to mine...
Miner Moe is attempting to mine...
Miner Moe successfully mined a block: 00cd72896f7a7a48a0e09e790ab2f3ec94b312b66a6b47965ae6775cc53a3be4
Miner Moe received a mining reward of 10


In [28]:
# Checks
print(f"Blockchain valid: {our_blockchain.validate_chain(our_network.public_keys)}")
print(f"Last added block signed by: {our_blockchain.chain[-1].signed_by}")
print(f"Proof of work direct check: {hash_criterion(our_blockchain.chain[-1].hash)}")
print(f"Is last block finalized: {our_blockchain.chain[-1].finalized}")

Blockchain valid: True
Last added block signed by: Miner Moe
Proof of work direct check: True
Is last block finalized: True


# Hashing

In [ ]:
# NOTE: there are automated tests for this
from src.hashing.sha2 import SHA256

In [25]:
# SHA-256 padding:
sample_message = "kutymutya"
binary_message = SHA256.convert_to_binary_string(sample_message)
binary_message = SHA256.add_padding(binary_message)
blocks = SHA256.split_to_blocks(binary_message)
blocks

['01101011011101010111010001111001011011010111010101110100011110010110000110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001001000']

In [27]:
SHA256.digest("hello") == '2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824'

True